In [2]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

from typing import List
from scipy.linalg import kron
from scipy.signal import find_peaks
from collections import defaultdict
from pennylane.optimize import AdamOptimizer

In [3]:
HAMILTONIAN = (-1.0523 * qml.Identity(0) @ qml.Identity(1) 
     + 0.3979 * qml.Identity(0) @ qml.PauliZ(1) 
     - 0.3979 * qml.PauliZ(0) @ qml.Identity(1) 
     - 0.0112 * qml.PauliZ(0) @ qml.PauliZ(1) 
     + 0.1809 * qml.PauliX(0) @ qml.PauliX(1))

# HAMILTONIAN = (
#     -12.533 * qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.Identity(3) @ qml.Identity(4) @ qml.Identity(5)
#     - 1.276 * qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.PauliZ(3) @ qml.Identity(4) @ qml.Identity(5)
#     + 0.627 * qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.Identity(3) @ qml.Identity(4) @ qml.Identity(5)
#     - 0.875 * qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.Identity(3) @ qml.PauliZ(4) @ qml.Identity(5)
#     + 0.452 * qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.PauliZ(3) @ qml.Identity(4) @ qml.Identity(5)
#     + 0.182 * qml.PauliX(0) @ qml.Identity(1) @ qml.PauliX(2) @ qml.Identity(3) @ qml.Identity(4) @ qml.Identity(5)
#     + 0.139 * qml.Identity(0) @ qml.PauliX(1) @ qml.Identity(2) @ qml.PauliX(3) @ qml.Identity(4) @ qml.Identity(5)
#     - 0.047 * qml.PauliY(0) @ qml.PauliY(1) @ qml.Identity(2) @ qml.Identity(3) @ qml.Identity(4) @ qml.Identity(5)
#     + 0.209 * qml.PauliZ(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.Identity(3) @ qml.PauliZ(4) @ qml.Identity(5)
#     - 0.154 * qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3) @ qml.Identity(4) @ qml.Identity(5)
#     + 0.198 * qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.PauliZ(3) @ qml.PauliZ(4) @ qml.PauliZ(5)
#     + 0.061 * qml.PauliX(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.Identity(3) @ qml.PauliX(4) @ qml.Identity(5)
#     - 0.027 * qml.Identity(0) @ qml.Identity(1) @ qml.PauliY(2) @ qml.Identity(3) @ qml.PauliY(4) @ qml.Identity(5)
#     + 0.118 * qml.PauliZ(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.PauliZ(3) @ qml.Identity(4) @ qml.PauliZ(5)
# ) 

In [4]:
def analytical_solution(hamiltonian):
    """
    Compute the analytical solution of the Hamiltonian.

    Args:
        - hamiltonian: express using the pennylane qml operations

    Returns:
        - eig_val: a list of eigenvalues
        - eig_vec: a list of eigenvectors
    """
    Hamiltonian_matrix = qml.matrix(hamiltonian)

    eig_vals, eig_vecs = np.linalg.eigh(Hamiltonian_matrix)

    # put the eigenvectors in the correct shape, i.e each row is an eigenvector
    eig_vecs = eig_vecs.transpose()

    # sort the eigenvalues and eigenvectors
    # for example, if eig_vals = [3, 1, 2], then eig_vals.argsort() will return [1, 2, 0]
    ordered_idxs = eig_vals.argsort()
    eig_vals = eig_vals[ordered_idxs]
    eig_vecs = eig_vecs[ordered_idxs]

    return eig_vals, eig_vecs

analytical_eig_vals, analytical_eig_vecs = analytical_solution(HAMILTONIAN)

In [5]:
def normalize(amps):
    return amps / np.linalg.norm(amps)

def get_uniform_superposition(eig_vecs):
    """
    Compute the uniform superposition of the eigenvectors.
    Args:
        - eig_vecs: a numpy array representing the eigenvectors

    Returns:
        - vec: a numpy array representing the uniform superposition
    """
    N = eig_vecs.shape[0]
    
    # Ensure there is at least one eigenvector
    if N == 0:
        return np.array([])

    # make sum and multiply by 1/sqrt(N) to normalize
    superposition = np.sum(eig_vecs, axis=0) / np.sqrt(N)
    
    # We apply the normalization again only to round up the values 0.999999.. to 1
    return normalize(superposition)

analytical_uniform_superposition = get_uniform_superposition(analytical_eig_vecs)

In [6]:
print("Analytical eigenvalues:",)
print(analytical_eig_vals)
print("\nAnalytical eigenvectors:")
print(analytical_eig_vecs)
print("\nAnalytical uniform superposition:")
print(analytical_uniform_superposition)

fidelities = [np.abs(np.dot(analytical_uniform_superposition, vec))**2 for vec in analytical_eig_vecs]
print("\nFidelities between the uniform superposition and the eigenvectors:")
print(fidelities)

Analytical eigenvalues:
[-1.85720199 -1.2444     -0.8826     -0.22499801]

Analytical eigenvectors:
[[ 0.        +0.j -0.99376135+0.j  0.11152752+0.j  0.        +0.j]
 [ 0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
 [-0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
 [ 0.        +0.j -0.11152752+0.j -0.99376135+0.j  0.        +0.j]]

Analytical uniform superposition:
[ 0.        +0.j -0.55264443+0.j -0.44111691+0.j -0.70710678+0.j]

Fidelities between the uniform superposition and the eigenvectors:
[0.25, 0.25, 0.25, 0.2500000000000001]


In [7]:
np.random.seed(0)  # Please don't change

def get_t_n(T_RMS, N, tol=0.01):
    """
    Generate a list of N positive random numbers from a normal distribution with a given RMS value.
    The function repeats the generation until the RMS of the generated numbers is within a tolerance of tol.

    Args:
        T_RMS: Desired RMS value.
        N: Number of random numbers to generate.
        tol: Tolerance for the RMS difference (default: 1).
        
    Returns:
        t_n: A list of N positive random numbers.
    """
    while True:
        t_n = np.abs(np.random.normal(loc=0, scale=T_RMS, size=N)).tolist()
        calculated_rms = np.sqrt(np.mean(np.square(t_n)))
        if abs(calculated_rms - T_RMS) < tol:
            print(f"Calculated RMS (T_RMS={T_RMS}): {calculated_rms}")
            return t_n

In [8]:
def plot_sample(Es,res,title='Rodeo Scan', threshold=0.22, legend=True):
    x_values = Es
    y_values = [res[E] for E in Es]

    peaks, _ = find_peaks(y_values,height=threshold)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(x_values, y_values, marker='o', linestyle='-', label='$P_n$')

    # Add vertical red lines at peaks
    for peak in peaks:
        plt.axvline(x=x_values[peak], color='red', linestyle='--', label=f'Peak at {x_values[peak]:.4f}')

    # Add vertical green line at correct eigenvalues
    for eigenvalue in analytical_eig_vals:
        plt.axvline(x=eigenvalue, color='green', linestyle='-.', label=f'Eigenvalue at {eigenvalue:.4f}')

    plt.title(title)
    plt.xlabel('E')
    plt.ylabel('$P_N$')
    plt.grid(True)
    if legend:
        plt.legend(loc='best')
    plt.show()

In [ ]:
N = 6
M = HAMILTONIAN.num_wires
T_RMS = 7

t_n = get_t_n(T_RMS, N)
dev = qml.device("default.qubit", wires=N+M)
initial_vec = analytical_uniform_superposition
# initial_vec = analytical_eig_vecs[1]

@qml.qnode(dev)
def circuit(E, t_n):
    for i in range(N):
        qml.Hadamard(wires=i)

    qml.StatePrep(initial_vec, wires=range(N, N + M))

    # Suzuki-Trotter
    for i in range(N):
        qml.ControlledQubitUnitary(
            qml.matrix(
                qml.TrotterProduct(
                    HAMILTONIAN, 
                    t_n[i], 
                    n=50, 
                    order=1
                )
            ),
            control_wires=[i],
            wires=range(N, N + M)
        )

    for i in range(N):
        qml.PhaseShift(-E * t_n[i], wires=[i])

    for i in range(N):
        qml.Hadamard(wires=i)

    return qml.probs(wires=range(N))

res =  defaultdict(list)
Es = np.linspace(-2, 0, 100)
for E in Es:
    Pn = circuit(E, t_n)[0]
    res[E] = Pn

In [10]:
# plot_sample(Es, res, threshold=0.10)